In [ ]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import pandas as pd
import time
import sys
import os


In [ ]:
dimension = 4

In [ ]:
def transform_log_to_sequence_example(logfile, dimension): 
    dimension=4
    data_names = ['x'+str(i) for i in range(dimension-1)]
    column_names = ['date', 'source'] + data_names
    perf_df = pd.read_csv('../logs/2019-01-17T12-36-59-4d-mdrnn.log', header=None, parse_dates=True, index_col=0, names=column_names)
    perf_df['t'] = perf_df.index
    perf_df.t = perf_df.t.diff()
    perf_df.t = perf_df.t.dt.total_seconds()
    perf_df = perf_df.dropna()
    # TODO: filter by interface/rnn source.
    return np.array(perf_df[['t']+data_names])


# Load up the performances
log_location = "../logs/"
log_file_ending = "-" + str(dimension) + "d-mdrnn.log"
log_arrays = []

for local_file in os.listdir(log_location):
    if local_file.endswith(log_file_ending):
        log = transform_log_to_sequence_example(log_location + local_file, dimension)
        log_arrays.append(log)
        print("Found:", local_file)

In [ ]:
### Save Performance Data in a compressed numpy file.
dataset_location = '../datasets/'
dataset_filename = 'training-dataset-'+ str(dimension) + 'd.npz'

# Input format is:
# 0. 1. 2. ... n.
# dt x1 x2 ... xn

raw_perfs = []

acc = 0
for l in log_arrays:
    acc += l.shape[0] * l.shape[1]
    raw = l.astype('float32') # dt, x_1, ... , x_n
    raw_perfs.append(raw)

print("total number of values:",acc)
print("total number of perfs in raw array:", len(raw_perfs))
raw_perfs = np.array(raw_perfs)
np.savez_compressed(dataset_location + dataset_filename, perfs=raw_perfs)
print("done saving:", dataset_location + dataset_filename)

In [ ]:
# Load tiny performance data from compressed file.
dataset_location = '../datasets/'
dataset_filename = 'training-dataset-'+ str(dimension) + 'd.npz'

with np.load(dataset_location + dataset_filename) as loaded:
    perfs = loaded['perfs']

print("Loaded perfs:", len(perfs))
print("Num touches:", np.sum([len(l) for l in perfs]))